In [1]:
MYHFKEY = "[PUT YOU HF KEY HERE]"

# Notebook initialization

## Install packages

In [13]:
!pip install langchain
!pip install pyPDF2
!pip install faiss-cpu
!pip install sentence_transformers
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=be615ecd82647861d5c17c11aac49fbb7925f5d168d5cfa942a9355249319017
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successf

## Import packages

In [3]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
import os
import warnings

warnings.filterwarnings('ignore')

In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = MYHFKEY
headers = {"Authorization": "Bearer " + MYHFKEY}

# First test with something which does not mean anything

Cf. https://huggingface.co/google/flan-t5-large

In [5]:
template = """ {question}"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)
llm=HuggingFaceHub(repo_id="google/flan-t5-large",
                   model_kwargs={"temperature":1e-10})
chain = LLMChain(llm=llm, prompt=prompt_template)

In [6]:
myQuery = "In how many movies benoit cayla played ?"
chain.run(myQuery)

'127'

Or more simply ;-)

In [17]:
llm(myQuery)

'127'

In [21]:
llm("What Benoit is doing now ?")

'He is a musician'

The answer is just totally wrong as this guy (me) never played in any movie LOL ... Apparently it existed another Benoit Cayla who was a musician, but this guy is definitely not the one we are interested in. However our LLM does not know him !

# Let's import the pdf and read the file

The file is on Github so this is its link (raw)

In [7]:
!wget https://raw.githubusercontent.com/datacorner/les-tutos-datacorner.fr/master/datasources/fakeinfos.pdf

--2023-11-05 06:55:43--  https://raw.githubusercontent.com/datacorner/les-tutos-datacorner.fr/master/datasources/fakeinfos.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47581 (46K) [application/octet-stream]
Saving to: ‘fakeinfos.pdf’

fakeinfos.pdf       100%[===================>]  46.47K  --.-KB/s    in 0.01s   

2023-11-05 06:55:43 (3.68 MB/s) - ‘fakeinfos.pdf’ saved [47581/47581]



In [8]:
newdata = PdfReader('/content/fakeinfos.pdf')
allContent = ""
for i, page in enumerate(newdata.pages):
  content = page.extract_text()
  if content:
    allContent += content

In [9]:
allContent

'Benoit Cayla is a famou s actor, he played in 70 movies and won 3 awards. Now he is playing with \ndata and some of t he most famous LLM models.  '

Chunk the document in several pieces ...

In [10]:
text_splitter = CharacterTextSplitter(separator="\n",
                                      chunk_size=200,
                                      chunk_overlap= 20,
                                      length_function= len)
finalAllContent = text_splitter.split_text(allContent)
finalAllContent

['Benoit Cayla is a famou s actor, he played in 70 movies and won 3 awards. Now he is playing with \ndata and some of t he most famous LLM models.']

# Embeddings / new content

Let's create some text embeddings now by using Hugging Face (HuggingFace sentence_transformers embedding models) / See https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub

In [14]:
embeddings = HuggingFaceEmbeddings()
#doc_result = embeddings.embed_documents(finalAllContent)

(…)851d5dd1af673670cdb299753/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)1af673670cdb299753/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)48e851d5dd1af673670cdb299753/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

(…)299753/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)851d5dd1af673670cdb299753/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)1af673670cdb299753/tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

(…)51d5dd1af673670cdb299753/train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)8e851d5dd1af673670cdb299753/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

# Makes the model more specific with the pdf inputs ...

Use the FAISS library to detect similarity between embeddings vectors / See https://ai.meta.com/tools/faiss/

*FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other. It solves limitations of traditional query search engines that are optimized for hash-based searches, and provides more scalable similarity search functions.*

The from_texts methods construct FAISS wrapper from raw documents.
This is a user friendly interface that:
* Embeds documents.
* Creates an in memory docstore
* Initializes the FAISS database



In [15]:
docSearch = FAISS.from_texts(finalAllContent, embeddings) # Meta Faiss vector store
newChain = load_qa_chain(llm, chain_type="stuff")
docs = docSearch.similarity_search(myQuery)

In [16]:
newChain.run(input_documents=docs, question=myQuery)

'70'

In [18]:
newChain.run(input_documents=docs, question="What Benoit is doing now ?")

'playing with data'

In [19]:
newChain.run(input_documents=docs, question="And how many awards did he won before playing with data ?")

'3'

**Yeah that's excactly the right result !!!**